In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.linalg as la
from sklearn.svm import SVC
from JDIP import JDIP
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale,LabelEncoder

In [2]:
def readData(sc,tg): 
    data = sio.loadmat('DomainAdaptationDatasets/OC10_vgg6/' + sc + '.mat')# source domain 
    Xs,ys = data['FTS'].astype(np.float64),data['LABELS'].ravel()
    ys = LabelEncoder().fit(ys).transform(ys).astype(np.float64)

    data = sio.loadmat('DomainAdaptationDatasets/OC10_vgg6/' + tg + '.mat')# target domain 
    Xt,yt = data['FTS'].astype(np.float64),data['LABELS'].ravel()
    yt = LabelEncoder().fit(yt).transform(yt).astype(np.float64)
    
    Xs = Xs / la.norm(Xs,axis=1,keepdims=True)
    Xt = Xt / la.norm(Xt,axis=1,keepdims=True)
    return Xs,ys,Xt,yt

domains = ['amazon','caltech','dslr','webcam']

In [3]:
estimator = LinearSVC()

tarSamp_per_class = 3

print 'details of each domain:'
for sc in domains:
    Xs,ys,Xt,yt = readData(sc,domains[0])
    print 'domain: ',sc,',', Xs.shape,',',  np.unique(ys,return_counts=True)

details of each domain:
domain:  amazon , (958L, 4096L) , (array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.]), array([ 92,  82,  94,  99, 100, 100,  99, 100,  94,  98], dtype=int64))
domain:  caltech , (1123L, 4096L) , (array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.]), array([151, 110, 100, 138,  85, 128, 133,  94,  87,  97], dtype=int64))
domain:  dslr , (157L, 4096L) , (array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.]), array([12, 21, 12, 13, 10, 24, 22, 12,  8, 23], dtype=int64))
domain:  webcam , (295L, 4096L) , (array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.]), array([29, 21, 31, 27, 27, 30, 43, 30, 27, 30], dtype=int64))


In [4]:
def source_partition(Xs,ys,samp_per_class,seed):
    Xs_tr,ys_tr = [],[]
    ys = ys.astype(np.int)
    for i in np.unique(ys):
        Xi,yi = Xs[ys==i],ys[ys==i]
        np.random.seed(seed)
        s = np.random.choice(len(yi),samp_per_class,replace=False)
        Xi_tr,yi_tr = Xi[s].tolist(),yi[s].tolist()
        Xs_tr.extend(Xi_tr),ys_tr.extend(yi_tr)

    return np.array(Xs_tr),np.array(ys_tr)

def target_partition(Xt,yt,samp_per_class,seed):
    Xt_tr,yt_tr,Xt_te,yt_te = [],[],[],[]
    yt = yt.astype(np.int)
    for i in np.unique(yt):
        Xi,yi = Xt[yt==i],yt[yt==i]
        np.random.seed(seed) 
        s1 = np.random.choice(len(yi),samp_per_class,replace=False)
        s2 = list(set(range(len(yi))) - set(s1))    
        Xi_tr,yi_tr,Xi_te,yi_te = Xi[s1].tolist(),yi[s1].tolist(),Xi[s2].tolist(),yi[s2].tolist()
        Xt_tr.extend(Xi_tr),yt_tr.extend(yi_tr),Xt_te.extend(Xi_te),yt_te.extend(yi_te)

    return np.array(Xt_tr),np.array(yt_tr),np.array(Xt_te),np.array(yt_te)

In [5]:
Xs,ys,Xt,yt = readData('amazon','dslr')
souSamp_per_class = 20 if sc == 'amazon' else 8
Xs_sub,ys_sub = source_partition(Xs,ys,souSamp_per_class,10)
Xtl,ytl,Xtu,ytu = target_partition(Xt,yt,tarSamp_per_class,10)
                
ytu0 = estimator.fit(np.vstack((Xs_sub,Xtl)),np.hstack((ys_sub,ytl))).predict(Xtu)
W0 = PCA().fit(np.vstack((Xs_sub,Xt))).components_.T
Ws,Wt = W0,W0
for k in range(10):
    Xt_,yt0_ = np.vstack((Xtl,Xtu)),np.hstack((ytl,ytu0))
    W = JDIP(dimension=100,lamda=1e-5,sigma=.5,maxiter=5,verbosity=0).fit(Xs_sub,ys_sub,Xt_,yt0_,Ws,Wt)
    Xs_sub_r,Xtl_r,Xtu_r = Xs_sub.dot(W[0]),Xtl.dot(W[1]),Xtu.dot(W[1])
        
    Xs_sub_r = Xs_sub_r / la.norm(Xs_sub_r,axis=1,keepdims=True)
    Xtl_r = Xtl_r / la.norm(Xtl_r,axis=1,keepdims=True)
    Xtu_r = Xtu_r / la.norm(Xtu_r,axis=1,keepdims=True)
                
    ytu0 = estimator.fit(np.vstack((Xs_sub_r,Xtl_r)),np.hstack((ys_sub,ytl))).predict(Xtu_r)
    #print k, '-th iter.', estimator.score(Xtu_r,ytu)
                                
print estimator.fit(np.vstack((Xs_sub_r,Xtl_r)),np.hstack((ys_sub,ytl))).score(Xtu_r,ytu) * 100.

89.7637795276
